In [42]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains
import pandas as pd
import time
import numpy as np
import re
from lxml import etree
import requests

In [ ]:
genecards_protein = pd.read_csv('D:/Thesis/Data/GeneCards-SearchResults2.csv')

In [ ]:
genecards_protein_only = pd.read_csv('GeneCards Protein Only.csv')

In [ ]:
genecards_protein_only = genecards_protein[genecards_protein['Category']=='Protein Coding']
genecards_protein_only

In [ ]:
genecards_protein_only

In [ ]:
driver = webdriver.Chrome('D:/chromedriver')

# GET UNIPROT ACCESSION FOR GENECARDS 

In [ ]:
driver = webdriver.Chrome('D:/chromedriver')
j=0
a=0
b=0
prot_accession = []
for i in genecards_protein_only[' GeneCards Link']:
    #masuk ke website genecards
    driver.get(i)
    try:
        print("Gene Symbol:", genecards_protein_only['Gene Symbol'].iloc[j])
        assert "No results found." not in driver.page_source
        #parse html
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        #ambil accession uniprot
        result=re.findall('://www.uniprot.org/uniprot/([\w\-\.]+)', str(soup.find_all("a", href=True)))
        print(result[0])
        prot_accession.append(result[0])
        j=j+1
        a=a+1
        print(prot_accession)
        print("restart...")
        driver.quit()
        driver = None
        driver = webdriver.Chrome('D:/chromedriver')
    except:
        print("No Accession Found")
        b=b+1
        continue
print("Total Prot with Accession:", a)
print("Total prot no accession:", b)        
prot_accession

In [ ]:
prot_accession2 = pd.DataFrame(prot_accession, columns = ['Accession'])
prot_accession2

In [ ]:
genecards_prot_only = pd.concat([genecards_protein_only.reset_index(),prot_accession2],ignore_index=True, axis=1)
genecards_prot_only.to_csv('GeneCards Protein Only2.csv')

In [ ]:
genecards_protein = pd.read_csv('GeneCards Protein Only2.csv')
genecards_protein.head()

# SUPERTARGET

In [ ]:
driver = webdriver.Chrome('D:/chromedriver')
driver.get("http://insilico.charite.de/supertarget/index.php?site=targets")
assert "SuperTarget" in driver.title

In [ ]:
accessionwithdrugsupertarget = pd.read_csv('D:\Thesis\Data\HasilScrapingSuperTargetNew1.csv')
accessionwithdrugsupertarget['Accession'].nunique()

In [ ]:
drugketinggalan = ['P51659', 'P27487', 'Q13291']

In [ ]:
drug = pd.DataFrame()
i=0
j=0
for accession in accessionwithdrugsupertarget['Accession'].unique():
    try:
        print("Accession:", accession)
        #Masukkan accession ke search bar accession
        accession_number = driver.find_element_by_name("accession_number_searchbox")
        accession_number.clear()
        accession_number.send_keys(accession)
        accession_number.send_keys(Keys.RETURN)
        time.sleep(10)
        assert "No results found." not in driver.page_source
        #Masuk ke bagian hasil dan click drugnya
        result = driver.find_element_by_class_name("row_odd")
        th = result.find_element_by_xpath('./td[8]')
        print("Drug Found, Clicking...")
        actions = ActionChains(driver)
        actions.move_to_element(result)
        actions.click(th)
        actions.perform()
        time.sleep(5)
        print("getting table...")
        #Ambil hasil drug dalam bentuk tabel
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        div=soup.select_one("div.tooltip")
        table=pd.read_html(str(div))
        print("merging...")
        #gabungin
        hasil = table[0]
        hasil['Accession'] = accession
        drug_id = re.findall('(?<=drug_details&amp;id\=)\d+', str(div.find_all("a", href=True)))
        hasil['CID']=drug_id
        filte = pd.DataFrame(hasil[['Drug','References','Accession','CID']])
        drug = pd.concat([drug,filte],ignore_index=True)
        print(drug)
        i=i+1
        print("restart...")
        driver.quit()
        driver = None
        driver = webdriver.Chrome('D:/chromedriver')
        driver.get("http://insilico.charite.de/supertarget/index.php?site=targets")
        assert "SuperTarget" in driver.title
    except:
        print("No Drug Found or Unknown Error")
        j=j+1
        continue
print("Total Accession with drug:", i)
print("Total Accession no drug:", j)        
drug

In [ ]:
drug2 = pd.DataFrame()
i=0
j=0
for accession in drugketinggalan:
    try:
        print("Accession:", accession)
        #Masukkan accession ke search bar accession
        accession_number = driver.find_element_by_name("accession_number_searchbox")
        accession_number.clear()
        accession_number.send_keys(accession)
        accession_number.send_keys(Keys.RETURN)
        time.sleep(10)
        assert "No results found." not in driver.page_source
        #Masuk ke bagian hasil dan click drugnya
        result = driver.find_element_by_class_name("row_odd")
        th = result.find_element_by_xpath('./td[8]')
        print("Drug Found, Clicking...")
        actions = ActionChains(driver)
        actions.move_to_element(result)
        actions.click(th)
        actions.perform()
        time.sleep(5)
        print("getting table...")
        #Ambil hasil drug dalam bentuk tabel
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        div=soup.select_one("div.tooltip")
        table=pd.read_html(str(div))
        print("merging...")
        #gabungin
        hasil = table[0]
        hasil['Accession'] = accession
        drug_id = re.findall('(?<=drug_details&amp;id\=)\d+', str(div.find_all("a", href=True)))
        hasil['CID']=drug_id
        filte = pd.DataFrame(hasil[['Drug','References','Accession','CID']])
        drug2 = pd.concat([drug2,filte],ignore_index=True)
        print(drug2)
        i=i+1
        print("restart...")
        driver.quit()
        driver = None
        driver = webdriver.Chrome('D:/chromedriver')
        driver.get("http://insilico.charite.de/supertarget/index.php?site=targets")
        assert "SuperTarget" in driver.title
    except:
        print("No Drug Found or Unknown Error")
        j=j+1
        continue
print("Total Accession with drug:", i)
print("Total Accession no drug:", j)        
drug2

In [ ]:
drugtotal = pd.concat([drug,drug2],ignore_index=True)
drugtotal.to_csv('drugtotalsupertarget.csv')

In [ ]:
drugtotalsupertarget = pd.read_csv('drugtotalsupertarget.csv')

In [ ]:
drugtotalsupertarget[drugtotalsupertarget['References']!='-'].to_csv('DrugTotalSuperTargetwithReferences.csv')

In [ ]:
accession_number = driver.find_element_by_name("accession_number_searchbox")
accession_number.clear()
accession_number.send_keys("Q9BYF1")
accession_number.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

In [ ]:
result = driver.find_element_by_class_name("row_odd")
th = result.find_element_by_xpath('./td[8]')

actions = ActionChains(driver)
actions.move_to_element(result)
actions.click(th)
actions.perform()

In [ ]:
table=pd.read_html(str(div))
hasil = table[0]
hasil['Accession'] = "Q9BYF1"
drug_id = re.findall('(?<=drug_details&amp;id\=)\d+', str(div.find_all("a", href=True)))
hasil['CID']=drug_id
drug = pd.DataFrame()
drug = hasil
drug

In [ ]:
# driver.quit()
# driver=None
# driver = webdriver.Chrome('D:/chromedriver')

In [ ]:
drug.to_csv('D:\Thesis\Data\HasilScrapingSuperTargetNew1.csv')

# DRUGBANK

In [ ]:
driver = webdriver.Chrome('D:/chromedriver')
driver.get("https://go.drugbank.com/")
assert "DrugBank" in driver.title

In [ ]:
driver.page_source

In [ ]:
target = driver.find_element_by_xpath("//label[@class='custom-radio'][2]")
target

actions = ActionChains(driver)
actions.move_to_element(target)
actions.click(target)
actions.perform()

In [ ]:
drug_target = driver.find_element_by_name("query")
drug_target.clear()
drug_target.send_keys("Q9NYK1")
drug_target.send_keys(Keys.RETURN)
assert "No results found." not in driver.page_source

In [ ]:
klikhasil = driver.find_element_by_xpath("//div[@class='unearth-search-hit my-1']")
klikhasil

actions = ActionChains(driver)
actions.move_to_element(klikhasil)
actions.click(klikhasil)
actions.perform()

In [ ]:
html=driver.page_source
soup=BeautifulSoup(html,'html.parser')
soup
div=soup.find('table', id="target-relations" )
print(div)

In [ ]:
table=pd.read_html(str(div))
hasil = table[0]
hasil

In [ ]:
list_drug_id

In [ ]:
#nyari compound idnya
link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sourceid/drugbank/DB04464/XML"
link

In [ ]:
# driver.switch_to.window(driver.window_handles[1])
driver.get(link)


In [ ]:
itsxml=driver.page_source
soupxml=BeautifulSoup(itsxml,'lxml')
soupxml
pubchemidtag = soupxml.findAll(re.compile("id_cid"))
pubchemid    = re.search('<pc-compoundtype_id_cid>(.+?)</pc-compoundtype_id_cid>',str(pubchemidtag)).group(1)
print(pubchemidtag)
pubchemid
# driver.close()

In [ ]:
driver = webdriver.Chrome('D:/chromedriver')
driver.get("https://go.drugbank.com/")
assert "DrugBank" in driver.title

In [ ]:
drugbank_drug=pd.DataFrame()
list_drug_id = []
k=0
l=0
for accession in genecards_protein['Accession']:
    try:
        driver.get("https://go.drugbank.com/")
        print("Accession:", accession)
        #masuk ke tab search target
        target = driver.find_element_by_xpath("//label[@class='custom-radio'][2]")
        actions = ActionChains(driver)
        actions.move_to_element(target)
        actions.click(target)
        actions.perform()
        #enter accession ke search bar
        drug_target = driver.find_element_by_name("query")
        drug_target.clear()
        drug_target.send_keys(accession)
        drug_target.send_keys(Keys.RETURN)
        #klik hasilnya
        klikhasil = driver.find_element_by_xpath("//div[@class='unearth-search-hit my-1']")
        actions = ActionChains(driver)
        actions.move_to_element(klikhasil)
        actions.click(klikhasil)
        actions.perform()
        time.sleep(5)
        #ambil tabel drugnya
        html=driver.page_source
        soup=BeautifulSoup(html,'html.parser')
        soup
        div=soup.find('table', id="target-relations" )
        #simpan tabelnya
        table=pd.read_html(str(div))
        hasil = table[0]
        hasil['Accession']=accession
        drugbank_drug=pd.concat([drugbank_drug,hasil], ignore_index=True)
        print(drugbank_drug)
        print("restarting...")
        k=k+1
    except:
        print("No Drug found or No Accession found in DrugBank")
        l=l+1
        continue
print("Total Accession with drug:", k)
print("Total Accession no drug:", l)        
drugbank_drug
        

In [ ]:
drugbank_drug = pd.read_csv('HasilScrapingDrugBankNew(not have PubChemID yet).csv')
drugbank_drug

In [ ]:
#cari compound id tiap drug
list_drug_id2 = []
number = 0
for drug in drugbank_drug['DrugBank ID']:
    print("Compound ke:", number)
    try:
        print('ID:', drug)
        #link ambil compound id pubchem
        link = "https://pubchem.ncbi.nlm.nih.gov/rest/pug/substance/sourceid/drugbank/" + drug + '/XML'
        #masuk ke link pubchem
        driver.get(link)
        #ambil pubchem id
        time.sleep(1)
        itsxml=driver.page_source
        soupxml=BeautifulSoup(itsxml,'lxml')
        soupxml
#       print(drug)
#       print(link)
        pubchemidtag = soupxml.findAll(re.compile("id_cid"))
        pubchemid    = re.search('<pc-compoundtype_id_cid>(.+?)</pc-compoundtype_id_cid>',str(pubchemidtag)).group(1)
#   print(pubchemid)
        #simpan ke dataframe
        list_drug_id2.append(pubchemid)
        print(list_drug_id2)
    except:
        print("No CID found or Unknown Error")
        list_drug_id2.append(None)
        continue
    number = number + 1
list_drug_id2

In [ ]:
drugbank_drug['PUBCHEM ID']= list_drug_id2

In [ ]:
drugbank_drug2 = drugbank_drug[drugbank_drug['PUBCHEM ID'].notnull()]
drugbank_drug2

In [ ]:
drugbank_drug2 = drugbank_drug[drugbank_drug['PUBCHEM ID'].notnull()]
# drugbank_drug2.head()
print(drugbank_drug.shape)
print(drugbank_drug2.shape)

In [ ]:
drugbank_drug.to_csv('E:\Thesis\Data\HasilScrapingDrugBank with PUBCHEMID and NONE.csv')

In [ ]:
drugbank_drug2.to_csv('D:\Thesis\Data\HasilScrapingDrugBankNew.csv')

In [ ]:
drugtotaldrugbank = pd.read_csv('D:\Thesis\Data\HasilScrapingDrugBankNew.csv')
drugtotaldrugbank

In [ ]:
drugtotaldrugbank[drugtotaldrugbank['Drug group'].str.contains("hello")]

# Scraping PUBCHEM ID data Herbal

In [2]:
data_herbal = pd.read_csv('D:\Thesis\Data\dataherbaltest.csv')
data_herbal

,Senyawa,1,2,3,4,5,6,7,8,9,...,872,873,874,875,876,877,878,879,880,881
0,Tetrahydroxyflavone,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,Rhamnetin,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
2,Psoralen,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,Orientanol_e,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
4,Laurotetanine,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
398,Majoroside,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
399,Loganic,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
400,Glucoputran,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
401,Isoforskolin,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0


In [49]:
data_herbal['Senyawa'][0]

'Tetrahydroxyflavone'

In [3]:
import pubchempy as pc
list_cid_herbal = []
i= 0
j = 0
for senyawa in data_herbal['Senyawa']:
    print("processing senyawa:", senyawa)
    try:
        list_cid_herbal.append(pc.get_cids(senyawa, 'name', 'compound', list_return='flat')[0])
        print('Senyawa retrieved, appending...')
        i=i+1
    except:
        print('Senyawa not found')
        list_cid_herbal.append(None)
        j=j+1
list_cid_herbal
        

processing senyawa: Tetrahydroxyflavone
Senyawa retrieved, appending...
processing senyawa: Rhamnetin
Senyawa retrieved, appending...
processing senyawa: Psoralen
Senyawa retrieved, appending...
processing senyawa: Orientanol_e
Senyawa not found
processing senyawa: Laurotetanine
Senyawa retrieved, appending...
processing senyawa: Hydroxyadoxoside
Senyawa not found
processing senyawa: Glucosinolate
Senyawa not found
processing senyawa: Ermanin
Senyawa retrieved, appending...
processing senyawa: ß-pinene
Senyawa not found
processing senyawa: ß-caryophyllene
Senyawa not found
processing senyawa: A-thujene
Senyawa not found
processing senyawa: A-terpinene
Senyawa retrieved, appending...
processing senyawa: A-pinene
Senyawa not found
processing senyawa: Y-mangostin
Senyawa not found
processing senyawa: Ylangene
Senyawa retrieved, appending...
processing senyawa: Y-cardinene
Senyawa not found
processing senyawa: Viridiflorene
Senyawa retrieved, appending...
processing senyawa: Vicenin
Senyaw

Senyawa retrieved, appending...
processing senyawa: 14-deoxy-11-oxoandrographolide
Senyawa retrieved, appending...
processing senyawa: 14-deoxy-11,12-didehydroandrographolide 2
Senyawa not found
processing senyawa: 14-deoxy-11,12-didehydroandrographolide
Senyawa retrieved, appending...
processing senyawa: Anisaldehyde
Senyawa retrieved, appending...
processing senyawa: Ascorbic acid
Senyawa retrieved, appending...
processing senyawa: Caprylic acid
Senyawa retrieved, appending...
processing senyawa: Citric acid
Senyawa retrieved, appending...
processing senyawa: Cyanidin 3-glucoside
Senyawa retrieved, appending...
processing senyawa: Cyanidin 3-xyloglucoside
Senyawa retrieved, appending...
processing senyawa: Cyanidin-3-sambubioside
Senyawa retrieved, appending...
processing senyawa: Delphinidin 3-sambubioside
Senyawa retrieved, appending...
processing senyawa: Delphinidin
Senyawa retrieved, appending...
processing senyawa: Delphinidin-3-glucoside
Senyawa retrieved, appending...
process

Senyawa retrieved, appending...
processing senyawa: Garcimangosone d
Senyawa retrieved, appending...
processing senyawa: Garcinone a
Senyawa retrieved, appending...
processing senyawa: Garcinone c
Senyawa retrieved, appending...
processing senyawa: Garcinone d
Senyawa retrieved, appending...
processing senyawa: Garcinone e
Senyawa retrieved, appending...
processing senyawa: Gartanin
Senyawa retrieved, appending...
processing senyawa: 5-hydroxyaloin a
Senyawa retrieved, appending...
processing senyawa: 7-hydroxy-8-o-methylaloin b
Senyawa retrieved, appending...
processing senyawa: Accemannan
Senyawa not found
processing senyawa: Aloemodin w-acetate
Senyawa not found
processing senyawa: Aloemodin
Senyawa not found
processing senyawa: Aloin
Senyawa retrieved, appending...
processing senyawa: Aloinoside b
Senyawa retrieved, appending...
processing senyawa: Arabinose
Senyawa retrieved, appending...
processing senyawa: Auxins
Senyawa not found
processing senyawa: B-sitosterol
Senyawa retriev

Senyawa retrieved, appending...
processing senyawa: Andrographolide type 4
Senyawa not found
processing senyawa: Andrographolide type 3
Senyawa not found
processing senyawa: Andrographolide type 2
Senyawa not found
processing senyawa: 14-deoxy-11,12-didehydroandrographolide2
Senyawa not found
processing senyawa: 11-hydroxymutakone
Senyawa not found
processing senyawa: Syringaresinol
Senyawa retrieved, appending...
processing senyawa: Amaranthine
Senyawa not found
processing senyawa: Methylthio
Senyawa retrieved, appending...
processing senyawa: arabinopyrano
Senyawa not found
processing senyawa: Peonidin-3
Senyawa not found
processing senyawa: Quercetin-3
Senyawa not found
processing senyawa: Theviridoside
Senyawa retrieved, appending...
processing senyawa: Sinigrin
Senyawa retrieved, appending...
processing senyawa: Caffeoylbeta
Senyawa not found
processing senyawa: Azadirachtin
Senyawa retrieved, appending...
processing senyawa: Hesperidine
Senyawa retrieved, appending...
processing 

[15233950,
 5281691,
 6199,
 None,
 31415,
 None,
 None,
 5352001,
 None,
 None,
 None,
 7462,
 None,
 None,
 6432119,
 None,
 10910653,
 3084407,
 9855795,
 64945,
 509268,
 14194109,
 177384,
 10281,
 8209,
 111037,
 11463,
 875,
 5316860,
 None,
 12921,
 5281,
 92231,
 124063,
 None,
 65373,
 None,
 None,
 5280460,
 5144,
 18818,
 5280805,
 124062,
 96191,
 92101,
 643684,
 None,
 5280459,
 5280459,
 5280343,
 None,
 72,
 1032,
 124025,
 6987,
 10639,
 11223782,
 None,
 358901,
 None,
 44256956,
 73386,
 8158,
 7463,
 637542,
 445638,
 985,
 19009,
 971,
 445639,
 10494,
 None,
 None,
 31289,
 125896,
 10477792,
 5491556,
 13989915,
 12398,
 5284507,
 None,
 None,
 None,
 None,
 442428,
 11005,
 5281672,
 31253,
 442756,
 151621,
 None,
 None,
 None,
 71308241,
 14162549,
 None,
 176079,
 14526924,
 None,
 102066427,
 85724901,
 7128,
 7127,
 637520,
 8815,
 5319518,
 65127,
 6478778,
 5281650,
 21672078,
 509267,
 10048103,
 5281647,
 44256979,
 159287,
 10416,
 525,
 444266,
 7333

In [9]:
data_herbal['CID'] = list_cid_herbal
data_herbal = data_herbal.dropna()
data_herbal.to_csv('D:\Thesis\Data\Dataherbaltest_with_cid.csv', index = False)

## GET InchiKey for Knapsack

In [12]:
data_herbal=pd.read_csv('D:\Thesis\Data\PubChemSAEDNN Herbal Compound Predict Result (Supplementary 5).csv')
data_herbal

,Compounds,Protein List,Probability,Total
0,Casuarinin,['PRKCA'],[1.0],1
1,Cycloeucalenol,['CSNK2A1'],[0.9999497],1
2,Fenchone,['CSNK2A1'],[0.99992883],1
3,Cycloeucalenone,['CSNK2A1'],[0.999928],1
4,Momordenol,['CSNK2A1'],[0.9999192],1
...,...,...,...,...
182,Phyllanthin,['PRKCA'],[0.5383905],1
183,Cinnamyl alcohol,['EGFR'],[0.530358],1
184,Benzaldehyde,['EGFR'],[0.5238626],1
185,Germacrene-d,['PRKCA'],[0.5071571],1


In [13]:
import pubchempy as pc
list_cid_herbal = []
i= 0
j = 0
for senyawa in data_herbal['Compounds']:
    print("processing senyawa:", senyawa)
    try:
        list_cid_herbal.append(pc.get_cids(senyawa, 'name', 'compound', list_return='flat')[0])
        print('Senyawa retrieved, appending...')
        i=i+1
    except:
        print('Senyawa not found')
        list_cid_herbal.append(None)
        j=j+1
list_cid_herbal


processing senyawa: Casuarinin
Senyawa retrieved, appending...
processing senyawa: Cycloeucalenol
Senyawa retrieved, appending...
processing senyawa: Fenchone
Senyawa retrieved, appending...
processing senyawa: Cycloeucalenone
Senyawa retrieved, appending...
processing senyawa: Momordenol
Senyawa retrieved, appending...
processing senyawa: B-sitosterol
Senyawa retrieved, appending...
processing senyawa: Campesterol
Senyawa retrieved, appending...
processing senyawa: Beta sitosterol
Senyawa retrieved, appending...
processing senyawa: 24-isopropyl cholesterol
Senyawa retrieved, appending...
processing senyawa: Momordicine 1
Senyawa not found
processing senyawa: Cycloarthenol
Senyawa not found
processing senyawa: (3beta,23e)-3-hydroxy-27-norcycloart-23-en-25-one
Senyawa retrieved, appending...
processing senyawa: Cucurbitane
Senyawa retrieved, appending...
processing senyawa: Diosgenin
Senyawa retrieved, appending...
processing senyawa: Gibberelins
Senyawa not found
processing senyawa: Au

Senyawa retrieved, appending...
processing senyawa: Garcimangosone d
Senyawa retrieved, appending...
processing senyawa: Ethyl cinnamate
Senyawa retrieved, appending...
processing senyawa: 7-hydroxy-8-o-methylaloin b
Senyawa retrieved, appending...
processing senyawa: Aloinoside b
Senyawa retrieved, appending...
processing senyawa: Kaempferol
Senyawa retrieved, appending...
processing senyawa: Aloemodin
Senyawa not found
processing senyawa: Amariin
Senyawa retrieved, appending...
processing senyawa: Cinnamic acid
Senyawa retrieved, appending...
processing senyawa: Rubiadin 1-methyl ether
Senyawa retrieved, appending...
processing senyawa: Physcione
Senyawa retrieved, appending...
processing senyawa: Morenone 1
Senyawa not found
processing senyawa: 2-methyl-4-hydroxy-5,7-dimethylanthraquinone
Senyawa not found
processing senyawa: P-coumaric acid
Senyawa retrieved, appending...
processing senyawa: N-cisferuloyltyramine
Senyawa not found
processing senyawa: N-transferuloltyramine
Senyawa 

[157395,
 101690,
 14525,
 21594790,
 85724901,
 222284,
 173183,
 222284,
 14079080,
 None,
 None,
 131752180,
 71306377,
 99474,
 None,
 None,
 442428,
 None,
 5281691,
 None,
 145925703,
 None,
 180535,
 5281321,
 441819,
 270061,
 441820,
 5281318,
 119242,
 5281315,
 5281319,
 10621,
 5281316,
 None,
 6616,
 None,
 73568,
 44445563,
 131752339,
 5281317,
 119287,
 None,
 64971,
 11240513,
 85203191,
 441822,
 10416810,
 85237911,
 656506,
 92101,
 5281320,
 5281322,
 125896,
 101761,
 131752341,
 131752164,
 None,
 None,
 519662,
 441005,
 12306048,
 None,
 5281323,
 14526924,
 91472,
 70678558,
 6432119,
 10225,
 None,
 None,
 None,
 15233950,
 None,
 None,
 10123,
 None,
 92231,
 44256956,
 44256979,
 None,
 None,
 44256919,
 10196837,
 None,
 92812,
 124025,
 44566353,
 177384,
 None,
 5281647,
 44256719,
 3084569,
 4023574,
 None,
 None,
 442664,
 73981613,
 326,
 5352001,
 5280805,
 None,
 None,
 151621,
 165558,
 7463,
 188289,
 12305761,
 None,
 None,
 5280459,
 5280459,
 6

In [14]:
data_herbal['CID'] = list_cid_herbal
data_herbal = data_herbal.dropna()
data_herbal.to_csv('D:\Thesis\Data\PubChem Predict Results with CID.csv', index = False)

In [15]:
import pubchempy as pcp

list_inchikey_herbal = []
i= 0
j = 0
for cid in data_herbal['CID']:
    print("processing senyawa with CID:", cid)
    try:
        c = pcp.Compound.from_cid(int(cid))
        c2 = c.to_dict(properties = ['inchikey'])
        list_inchikey_herbal.append(c2['inchikey'])
        print('Senyawa retrieved, appending...', c2['inchikey'])
        i=i+1
    except:
        print('InchiKey not found')
        list_inchikey_herbal.append(None)
        j=j+1
list_inchikey_herbal

processing senyawa with CID: 157395.0
Senyawa retrieved, appending... MMQXBTULXAEKQE-UHFFFAOYSA-N
processing senyawa with CID: 101690.0
Senyawa retrieved, appending... HUNLTIZKNQDZEI-PGFZVWMDSA-N
processing senyawa with CID: 14525.0
Senyawa retrieved, appending... LHXDLQBQYFFVNW-UHFFFAOYSA-N
processing senyawa with CID: 21594790.0
Senyawa retrieved, appending... NFRXSIOHGADFRG-MEMZBLDGSA-N
processing senyawa with CID: 85724901.0
Senyawa retrieved, appending... MEWYFWRPPPAWSI-UHFFFAOYSA-N
processing senyawa with CID: 222284.0
Senyawa retrieved, appending... KZJWDPNRJALLNS-VJSFXXLFSA-N
processing senyawa with CID: 173183.0
Senyawa retrieved, appending... SGNBVLSWZMBQTH-PODYLUTMSA-N
processing senyawa with CID: 222284.0
Senyawa retrieved, appending... KZJWDPNRJALLNS-VJSFXXLFSA-N
processing senyawa with CID: 14079080.0
Senyawa retrieved, appending... MVDZEJXAUJRFGW-XKTYQPJFSA-N
processing senyawa with CID: 131752180.0
Senyawa retrieved, appending... WNTCMDLOMSFFPE-VQHVLOKHSA-N
processing s

Senyawa retrieved, appending... OIUBYZLTFSLSBY-HMGRVEAOSA-N
processing senyawa with CID: 10494.0
Senyawa retrieved, appending... MIJYXULNPSFWEK-GTOFXWBISA-N
processing senyawa with CID: 64945.0
Senyawa retrieved, appending... WCGUUGGRBIKTOS-GPOJBZKASA-N
processing senyawa with CID: 5281643.0
Senyawa retrieved, appending... OVSQVDMCBVZWGM-DTGCRPNFSA-N
processing senyawa with CID: 5280804.0
Senyawa retrieved, appending... OVSQVDMCBVZWGM-QSOFNFLRSA-N
processing senyawa with CID: 91354.0
Senyawa retrieved, appending... ITYNGVSTWVVPIC-UHFFFAOYSA-N
processing senyawa with CID: 5281855.0
Senyawa retrieved, appending... AFSDNFLWKVMVRB-UHFFFAOYSA-N
processing senyawa with CID: 137199997.0
Senyawa retrieved, appending... VSPBJCAGAJBGKS-UHFFFAOYSA-N
processing senyawa with CID: 5281697.0
Senyawa retrieved, appending... JVXZRQGOGOXCEC-UHFFFAOYSA-N
processing senyawa with CID: 12303220.0
Senyawa retrieved, appending... YTMNONATNXDQJF-UHFFFAOYSA-N
processing senyawa with CID: 31415.0
Senyawa retriev

['MMQXBTULXAEKQE-UHFFFAOYSA-N',
 'HUNLTIZKNQDZEI-PGFZVWMDSA-N',
 'LHXDLQBQYFFVNW-UHFFFAOYSA-N',
 'NFRXSIOHGADFRG-MEMZBLDGSA-N',
 'MEWYFWRPPPAWSI-UHFFFAOYSA-N',
 'KZJWDPNRJALLNS-VJSFXXLFSA-N',
 'SGNBVLSWZMBQTH-PODYLUTMSA-N',
 'KZJWDPNRJALLNS-VJSFXXLFSA-N',
 'MVDZEJXAUJRFGW-XKTYQPJFSA-N',
 'WNTCMDLOMSFFPE-VQHVLOKHSA-N',
 'ZYZJWAJOTPNVPI-ZVBSCDOUSA-N',
 'WQLVFSAGQJTQCK-VKROHFNGSA-N',
 'DFPMSGMNTNDNHN-ZPHOTFPESA-N',
 'JGUZGNYPMHHYRK-UHFFFAOYSA-N',
 'OHVYCELRHUJPFH-CZVRGWHYSA-N',
 'ITMUUFDDBRYVNJ-VOKXYEOFSA-N',
 'NISPVUDLMHQFRQ-MKIKIEMVSA-N',
 'FBGLZDYMEULGSX-IDZQOQCMSA-N',
 'FBGLZDYMEULGSX-UHFFFAOYSA-N',
 'PIGAXYFCLPQWOD-ILFSFOJUSA-N',
 'SRPHMISUTWFFKJ-QJNWWGCFSA-N',
 'XCDQFROEGGNAER-PFOIMGGJSA-N',
 'IHTCCHVMPGDDSL-IVNGUWCNSA-N',
 'NDYMQXYDSVBNLL-MUYMLXPFSA-N',
 'QUQPHWDTPGMPEX-QJBIFVCTSA-N',
 'IXQKXEUSCPEQRD-DKRGWESNSA-N',
 'CRPUJAZIXJMDBK-UHFFFAOYSA-N',
 'TUSDEZXZIZRFGC-XIGLUPEJSA-N',
 'JYKQEJLWLRMCRC-KXCQKKMCSA-N',
 'VTWHHBJKZZJDQM-OQLLNIDSSA-N',
 'DGIGXLXLGBAJJN-TUOUHCSQSA-N',
 'MBYLRW

In [16]:
data_herbal['InchiKey']=list_inchikey_herbal
data_herbal.to_csv('D:\Thesis\Data\Dataherbaltest_withinchikey.csv', index = False)

<ipython-input-16-af2f2b560390>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_herbal['InchiKey']=list_inchikey_herbal


In [17]:

# target = driver.find_element_by_xpath("//td[@class='d1'][1]/a")
# actions = ActionChains(driver)
# actions.move_to_element(target)
# actions.click(target)
# actions.perform()
# driver.switch_to.window(driver.window_handles[1])
# driver.page_source
# html=driver.page_source
# soup=BeautifulSoup(html,'html.parser')
# soup
# div=soup.find_all( "table" , class_ = "org" )
# table=pd.read_html(str(div))
# hasil = table[0]
# hasil

'<html><!-- <head> --><!-- NBDC Markup START --><head vocab="http://schema.org/" typeof="BiologicalDatabaseEntry">\n<!-- NBDC Markup END -->\n\n<title>KNApSAcK Metabolite Information - C00004542</title>\n<meta http-equiv="Content-Type" content="text/html">\n<meta http-equiv="Content-Style-Type" content="text/cs">\n<meta http-equiv="Pragma" content="no-cache">\n<meta http-equiv="Cache-Control" content="no-cache">\n<meta name="description" content="C00004542">\n<link rel="stylesheet" type="text/css" href="css/top.css" media="all">\n<link rel="stylesheet" type="text/css" href="css/info.css" media="all">\n</head><body onload="">\n\n<div id="my_body">\n  <div id="my_header">\n      <p class="hm"><a href="top.php" style="text-decoration:none" target="_blank">\n      <img src="images/nap2.gif" border="0"> </a></p>\n  </div>\n  <div id="my_main">\n    <div id="my_navigation">\n\n    </div>\n    <div id="my_contents">\n          <br>\n<!--\t  input word = <font class="iw">C00004542</font><br><b